In [1]:
%load_ext autoreload
%autoreload 2 

### Cache data - price, prododuct data etc
suppoedly it caches itself in data.py, but that logic broke

In [2]:
from data.data import PriceData, ProductData, BenchmarkData, START_DATE, Benchmarks

In [3]:
%%timeit
benchmark_data = BenchmarkData()
tickers = benchmark_data.get_constituents(Benchmarks.SP500)
len(tickers)

Scraped 503 tickers for SP500 from Wikipedia
Cached 1 entries


503

In [4]:
%%timeit
product_data = ProductData()
data = product_data.get_data(tickers=tickers[:10])
len(data)

10

In [5]:
%%timeit
price_data = PriceData()
data = price_data.get_data(tickers[:10], start_date=START_DATE, end_date='2025-06-01')
len(data)

3

### Backtesting

In [4]:
# first create ConstraintsConfig and PortfolioConfig
from backtesting.backtest import Backtest
from backtesting.scenarios import Scenario
from portfolio.constraints import ConstraintsConfig
from portfolio.portfolio import CapitalGrowthFrequency, PortfolioConfig
from strategies.strategy import StrategyTypes
from data.data import Benchmarks
verbose = True

constraints_config = ConstraintsConfig(
    long_only=True,
    cash_pct=0.0,
    max_long_trades=0.5,
    max_short_trades=0.5,
    max_buy_size=0.3,
)
portfolio_config = PortfolioConfig(
    initial_capital=100_000,
    initial_holdings={},
    new_capital_growth_amt=10000,
    capital_growth_freq=CapitalGrowthFrequency.MONTHLY.value,
)

# then create scenario, note portfolio is created within scenario
scenario = Scenario(
    name="test backtest function",
    start_date="2015-01-01",
    end_date="2025-05-01",
    benchmark=Benchmarks.SP500,
    constraints=constraints_config,
    portfolio_config=portfolio_config,
    verbose=verbose,
    portfolio_name="sp500_10yr_long_only_macd_rsi"
)

# then create strategies
strategies = {StrategyTypes.MACD_CROSSOVER: True, StrategyTypes.RSI_CROSSOVER: True}
scenario.set_strategies(strategies)


# then create backtest
backtest = Backtest(scenario, verbose=True)

# then run test!
backtest.run_batch()

Portfolio setup: {
    "initial_capital": 100000,
    "initial_holdings": {},
    "initial_value": 0,
    "capital_growth_freq": "M",
    "new_capital_growth_pct": 0,
    "new_capital_growth_amt": 10000,
    "allocation_method": "equal",
    "excluded_sectors": [],
    "included_countries": [
        "United States"
    ],
    "min_market_cap": 0,
    "max_market_cap": Infinity
}
Strategies: {
    "macd_x": true,
    "rsi_x": true
}
Backtest starting... swoosh!
Universe size: 480
Total trading days: 2598
Total strategies: 2
Starting in 2015-01-01
Ending in 2025-05-01


Backtesting by strategy: 100%|██████████| 2/2 [00:24<00:00, 12.42s/strategy]


Backtest completed!


In [1]:
backtest.generate_report(filename="sample_report")

NameError: name 'backtest' is not defined

### Gridsearch

In [1]:
from backtesting.grid_search import GridSearch
from backtesting.scenarios import Scenario
from data.data import Benchmarks
from example.config import DEFAULT_CONSTRAINTS, DEFAULT_PORTFOLIO_SETUP
from strategies.strategy import StrategyTypes

# first create grid search params
grid_search_params =[
    {StrategyTypes.MACD_CROSSOVER: True},
    {StrategyTypes.MACD_CROSSOVER: True, StrategyTypes.BOLLINGER_BANDS: False},
    {StrategyTypes.RSI_CROSSOVER: True, StrategyTypes.Z_SCORE_MEAN_REVERSION: True},
    {
        StrategyTypes.MACD_CROSSOVER: True,
        StrategyTypes.RSI_CROSSOVER: True,
        StrategyTypes.BOLLINGER_BANDS: True,
        StrategyTypes.Z_SCORE_MEAN_REVERSION: True,
    },
]

# or can automatically generate grid search params
strategy_list = [StrategyTypes.MACD_CROSSOVER, StrategyTypes.RSI_CROSSOVER, StrategyTypes.BOLLINGER_BANDS, StrategyTypes.Z_SCORE_MEAN_REVERSION]

# then create scenario
scenario = Scenario(
    name="test grid search function",
    start_date="2020-01-01",
    end_date="2022-01-01",    
    constraints=DEFAULT_CONSTRAINTS,
    portfolio_config=DEFAULT_PORTFOLIO_SETUP,
    benchmark=Benchmarks.SP500,
    verbose=False,
)

# then create grid search
gs = GridSearch(base_scenario=scenario, max_workers=10, verbose=True)
# gs.set_grid_params(grid_search_params)
gs.set_grid_params(strategy_list, max_signal=3, max_filter=2, min_signal=1)

# then run grid search
results = gs.run()

Generated 60 grid parameter combinations
Running grid search with 60 parameter combinations...


Grid search progress: 100%|██████████| 60/60 [01:08<00:00,  1.15s/it]


Grid search completed! Found 60 valid results.


In [2]:
# can print the run schedule
print(gs.get_grid_search_schedule())

print()
# then print results
gs.results_to_text()

   grid_num                                  param_name
1         1                               **Pos: macd_x
4        10                **Pos: b_bands **Neg: macd_x
21       11                          **Pos: macd_x || z
11       12                      **Pos: macd_x **Neg: z
23       13                      **Pos: z **Neg: macd_x
8        14                     **Pos: rsi_x || b_bands
14       15                 **Pos: rsi_x **Neg: b_bands
7        16                 **Pos: b_bands **Neg: rsi_x
15       17                           **Pos: rsi_x || z
12       18                       **Pos: rsi_x **Neg: z
19       19                       **Pos: z **Neg: rsi_x
2         2                                **Pos: rsi_x
10       20                         **Pos: b_bands || z
13       21                     **Pos: b_bands **Neg: z
20       22                     **Pos: z **Neg: b_bands
16       23           **Pos: macd_x || rsi_x || b_bands
29       24       **Pos: macd_x || rsi_x **Neg: 

In [3]:
gs.results_to_csv("poc_gs_results")